## Exericse 9

## Dataset Cloning

In [2]:
# Clone the chitchat dataset from GitHub
!git clone https://github.com/BYU-PCCL/chitchat-dataset.git

# Navigate into the folder and list files
!ls chitchat-dataset


fatal: destination path 'chitchat-dataset' already exists and is not an empty directory.
chitchat_dataset  examples  Makefile	 pyproject.toml
CONTRIBUTING.md   LICENSE   poetry.lock  README.md


In [4]:
import json, os

dataset_path = "/kaggle/working/chitchat-dataset/chitchat_dataset/dataset.json"

with open(dataset_path, "r", encoding="utf-8") as f:
    dataset = json.load(f)

print("✅ Loaded dataset with", len(dataset), "entries.")
print("Top-level type:", type(dataset))


✅ Loaded dataset with 7168 entries.
Top-level type: <class 'dict'>


## Data Extraction

In [6]:
# Extract all conversation objects from dictionary values
conversations = list(dataset.values())
print("✅ Extracted", len(conversations), "conversation threads.")
print("Sample keys in first conversation:", list(conversations[0].keys()))


✅ Extracted 7168 conversation threads.
Sample keys in first conversation: ['messages', 'ratings', 'start', 'prompt']


## Data Cleaning


In [7]:
import re

def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r"[^a-zA-Z0-9\s,.!?']", "", text)
    return text

pairs = []

for conv in conversations:
    if "messages" not in conv:
        continue
    messages = conv["messages"]

    # Flatten messages across sublists
    all_msgs = []
    for group in messages:
        for msg in group:
            if "text" in msg:
                all_msgs.append(clean_text(msg["text"]))

    # Now create consecutive pairs (previous msg -> next msg)
    for i in range(len(all_msgs) - 1):
        context = all_msgs[i]
        response = all_msgs[i + 1]
        if context and response:
            pairs.append((context, response))

print(f"✅ Extracted {len(pairs)} (context, response) pairs from {len(conversations)} conversations.")
print("Sample pairs:")
for i in range(5):
    print(f"{i+1}. Q: {pairs[i][0]} → A: {pairs[i][1]}")


✅ Extracted 250635 (context, response) pairs from 7168 conversations.
Sample pairs:
1. Q: hello → A: how are you doing today?
2. Q: how are you doing today? → A: whats up md
3. Q: whats up md → A: im doing good
4. Q: im doing good → A: how are you doing?
5. Q: how are you doing? → A: im alright, i just took a nap. but it was one of those naps that doesnt help anything. it just makes everything worse and you question all your life choices


## Tokenization Setup

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

MAX_LEN = 20
VOCAB_SIZE = 10000

input_texts = [p[0] for p in pairs]
target_texts = ["<start> " + p[1] + " <end>" for p in pairs]

tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters="")
tokenizer.fit_on_texts(input_texts + target_texts)

input_seqs = tokenizer.texts_to_sequences(input_texts)
target_seqs = tokenizer.texts_to_sequences(target_texts)

encoder_input = pad_sequences(input_seqs, maxlen=MAX_LEN, padding="post")
decoder_input = pad_sequences(target_seqs, maxlen=MAX_LEN, padding="post")

enc_train, enc_test, dec_train, dec_test = train_test_split(
    encoder_input, decoder_input, test_size=0.1, random_state=42
)

print(f"✅ Tokenization done: {enc_train.shape[0]} train samples, {enc_test.shape[0]} test samples.")


2025-11-05 14:20:20.077667: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762352420.279295     152 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762352420.340773     152 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ Tokenization done: 225571 train samples, 25064 test samples.


## Vocabulary Creation

In [9]:
import numpy as np

print("Encoder input shape:", encoder_input.shape)
print("Decoder input shape:", decoder_input.shape)
print("Vocabulary size:", len(tokenizer.word_index))

# Quick reverse check
reverse_vocab = {v: k for k, v in tokenizer.word_index.items()}

# View a random training example
idx = np.random.randint(0, len(input_texts))
print("\n🔹 Example:")
print("Input:", input_texts[idx])
print("Target:", target_texts[idx])


Encoder input shape: (250635, 20)
Decoder input shape: (250635, 20)
Vocabulary size: 71407

🔹 Example:
Input: i wanted an ipad, sure, but it was more . . . it was more that i'd told myself i was winning that ipad.
Target: <start> if i couldn't do this, something that's relatively simple, how on earth am i supposed to keep my bigger commitments? <end>


## Model and Training

In [32]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, AdditiveAttention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np, pickle

# --- Config ---
SAMPLE_SIZE = 30000
MAX_LEN = 30
VOCAB_LIMIT = 15000
EMBED_DIM = 128
LATENT_DIM = 256
EPOCHS = 25
BATCH_SIZE = 64
LR = 1e-3

# --- Helper: clip tokens to vocab limit ---
def clip_tokens(sequences, vocab_size):
    return [[min(token, vocab_size - 1) for token in seq] for seq in sequences]

# --- Prepare subset ---
enc_train_sub = np.array(clip_tokens(enc_train[:SAMPLE_SIZE].tolist(), VOCAB_LIMIT), dtype=np.int32)
dec_train_sub = np.array(clip_tokens(dec_train[:SAMPLE_SIZE].tolist(), VOCAB_LIMIT), dtype=np.int32)
VOCAB_SIZE = VOCAB_LIMIT

print(f"✅ Using {SAMPLE_SIZE} samples, vocab size {VOCAB_SIZE}, maxlen {MAX_LEN}")

# --- Encoder ---
encoder_inputs = Input(shape=(None,), name="encoder_input")
enc_emb = Embedding(VOCAB_SIZE, EMBED_DIM)(encoder_inputs)
encoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True, name="encoder_lstm")
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# --- Decoder ---
decoder_inputs = Input(shape=(None,), name="decoder_input")
dec_emb = Embedding(VOCAB_SIZE, EMBED_DIM)(decoder_inputs)
decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# --- Additive Attention ---
attention = AdditiveAttention(name="additive_attention")
attn_out = attention([decoder_outputs, encoder_outputs])
decoder_combined = Concatenate(axis=-1)([decoder_outputs, attn_out])

# --- Dense projection ---
decoder_dense = Dense(VOCAB_SIZE, activation="softmax", name="decoder_dense")
decoder_outputs = decoder_dense(decoder_combined)

# --- Model ---
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(learning_rate=LR),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

# --- Shift decoder sequences for teacher forcing ---
decoder_target = dec_train_sub[:, 1:]
decoder_input_trimmed = dec_train_sub[:, :-1]
decoder_target_exp = decoder_target[..., None]

print("Shapes:")
print("enc_train_sub:", enc_train_sub.shape)
print("decoder_input_trimmed:", decoder_input_trimmed.shape)
print("decoder_target:", decoder_target.shape)

# --- Callbacks: learning-rate scheduler + early stopping ---
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=2, verbose=1, min_lr=1e-5
)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True, verbose=1
)

# --- Sanity check small forward pass ---
_ = model.predict([enc_train_sub[:2], decoder_input_trimmed[:2]])

# --- Train ---
history = model.fit(
    [enc_train_sub, decoder_input_trimmed],
    decoder_target_exp,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
    callbacks=[lr_scheduler, early_stop],
    verbose=1
)

# --- Save ---
model.save("/kaggle/working/chatbot_seq2seq_optimized.h5")
with open("/kaggle/working/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ Optimized training finished and model saved successfully.")


✅ Using 30000 samples, vocab size 15000, maxlen 30


Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_18        │ (None, None, 128) │  1,920,000 │ encoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_19        │ (None, None, 128) │  1,920,000 │ decoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, None,     │    394,240 │ embedding_18[0][… │
│                     │ 256), (None,      │            │                   │
│                     │ 256), (None,      │            │                   │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │    394,240 │ embedding_19[0][… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256), (None,      │            │ encoder_lstm[0][… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additive_attention  │ (None, None, 256) │        256 │ decoder_lstm[0][… │
│ (AdditiveAttention) │                   │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_12      │ (None, None, 512) │          0 │ decoder_lstm[0][… │
│ (Concatenate)       │                   │            │ additive_attenti… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None,      │  7,695,000 │ concatenate_12[0… │
│ (Dense)             │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,323,736 (47.01 MB)

 Trainable params: 12,323,736 (47.01 MB)

 Non-trainable params: 0 (0.00 B)

Shapes:
enc_train_sub: (30000, 20)
decoder_input_trimmed: (30000, 19)
decoder_target: (30000, 19)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
Epoch 1/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 40s 84ms/step - accuracy: 0.5444 - loss: 3.7541 - val_accuracy: 0.5733 - val_loss: 2.7539 - learning_rate: 0.0010
Epoch 2/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 36s 85ms/step - accuracy: 0.5859 - loss: 2.6529 - val_accuracy: 0.5969 - val_loss: 2.5485 - learning_rate: 0.0010
Epoch 3/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 36s 85ms/step - accuracy: 0.6100 - loss: 2.4195 - val_accuracy: 0.6021 - val_loss: 2.4461 - learning_rate: 0.0010
Epoch 4/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 36s 85ms/step - accuracy: 0.6145 - loss: 2.3124 - val_accuracy: 0.6057 - val_loss: 2.3991 - learning_rate: 0.0010
Epoch 5/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 36s 85ms/step - accuracy: 0.6192 - loss: 2.2290 - val_accuracy: 0.6085 - val_loss: 2.3667 - learning_rate: 0.0010
Epoch 6/25
422/422 ━━━━━━━━━━━━━━━━━━━━ 36s 85ms/step - accuracy: 0.6227 - loss: 2.1663 - val

## Sample Try Out

In [33]:
# --- Combined Inference, Decoding, and Evaluation Cell ---
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np, pickle

# --- Load model and tokenizer ---
model = load_model("/kaggle/working/chatbot_seq2seq_optimized.h5", compile=False)
with open("/kaggle/working/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

LATENT_DIM = 256
MAX_LEN = 30

print(f"✅ Model loaded with {len(model.layers)} layers")
print("Available layers:", [layer.name for layer in model.layers])

# --- Identify embedding layers dynamically ---
embedding_layers = [l for l in model.layers if "embedding" in l.name]
encoder_emb_layer = embedding_layers[0]
decoder_emb_layer = embedding_layers[1]

# --- Encoder inference model ---
encoder_inputs = model.input[0]  # first input (from training)
encoder_outputs, state_h_enc, state_c_enc = model.get_layer("encoder_lstm").output
encoder_model = tf.keras.Model(encoder_inputs, [encoder_outputs, state_h_enc, state_c_enc])

# --- Decoder inference model ---
decoder_inputs = model.input[1]  # second input (from training)
decoder_lstm = model.get_layer("decoder_lstm")
attn_layer = model.get_layer("additive_attention")
concat_layer = [l for l in model.layers if "concatenate" in l.name][0]  # auto-detect concatenate layer
dense_layer = model.get_layer("decoder_dense")

# Define new inputs for decoder states and encoder outputs
decoder_state_input_h = Input(shape=(LATENT_DIM,))
decoder_state_input_c = Input(shape=(LATENT_DIM,))
encoder_out_input = Input(shape=(None, LATENT_DIM))

# Rebuild the decoder computation graph for inference
dec_emb_infer = decoder_emb_layer(decoder_inputs)
dec_outputs, state_h_dec, state_c_dec = decoder_lstm(
    dec_emb_infer, initial_state=[decoder_state_input_h, decoder_state_input_c]
)
attn_out_inf = attn_layer([dec_outputs, encoder_out_input])
dec_combined_inf = concat_layer([dec_outputs, attn_out_inf])
dec_preds = dense_layer(dec_combined_inf)

decoder_model = tf.keras.Model(
    [decoder_inputs, encoder_out_input, decoder_state_input_h, decoder_state_input_c],
    [dec_preds, state_h_dec, state_c_dec]
)

print("✅ Encoder and Decoder inference models rebuilt successfully.")

# --- Token index helpers ---
reverse_word_index = {v: k for k, v in tokenizer.word_index.items()}
START_TOKEN = tokenizer.word_index.get("<start>", 1)
END_TOKEN = tokenizer.word_index.get("<end>", 2)

# --- Decoding function ---
def decode_sequence(input_text):
    input_seq = tokenizer.texts_to_sequences([input_text.lower()])
    input_seq = pad_sequences(input_seq, maxlen=MAX_LEN, padding='post')

    enc_outs, state_h, state_c = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = START_TOKEN

    decoded_words = []
    for _ in range(MAX_LEN):
        preds, h, c = decoder_model.predict([target_seq, enc_outs, state_h, state_c])
        next_token = np.argmax(preds[0, -1, :])
        if next_token == END_TOKEN:
            break
        word = reverse_word_index.get(next_token, '')
        if word:
            decoded_words.append(word)
        target_seq[0, 0] = next_token
        state_h, state_c = h, c

    return ' '.join(decoded_words)

# --- Evaluate chatbot on sample inputs ---
test_inputs = [
    "hi how are you",
    "what are you doing today",
    "do you like photography",
    "tell me about sketch app",
    "that's terrible",
    "what's your favorite color",
    "how's the weather"
]

print("\n🗣️ Chatbot responses:\n")
for inp in test_inputs:
    print(f"User: {inp}")
    print(f"Bot:  {decode_sequence(inp)}\n")


✅ Model loaded with 9 layers
Available layers: ['encoder_input', 'decoder_input', 'embedding_18', 'embedding_19', 'encoder_lstm', 'decoder_lstm', 'additive_attention', 'concatenate_12', 'decoder_dense']
✅ Encoder and Decoder inference models rebuilt successfully.

🗣️ Chatbot responses:

User: hi how are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Bot:  i am doing well!

User: what are you doing today
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Bot:  i don't know how to do it

User: do you like photography
1

## Evaluation with BLEU

In [34]:
# --- Fully Silent Final Evaluation: BLEU & Sample Outputs ---
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
import tensorflow as tf
import os

# --- Silence TF logs completely ---
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.get_logger().setLevel("ERROR")

smoothie = SmoothingFunction().method4

# --- Overwrite decode_sequence() to suppress progress ---
def decode_sequence(input_text):
    input_seq = tokenizer.texts_to_sequences([input_text.lower()])
    input_seq = pad_sequences(input_seq, maxlen=MAX_LEN, padding='post')

    enc_outs, state_h, state_c = encoder_model.predict(input_seq, verbose=0)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = START_TOKEN

    decoded_words = []
    for _ in range(MAX_LEN):
        preds, h, c = decoder_model.predict([target_seq, enc_outs, state_h, state_c], verbose=0)
        next_token = np.argmax(preds[0, -1, :])
        if next_token == END_TOKEN:
            break
        word = reverse_word_index.get(next_token, '')
        if word:
            decoded_words.append(word)
        target_seq[0, 0] = next_token
        state_h, state_c = h, c

    return ' '.join(decoded_words)

# --- Evaluation ---
num_eval = 250
refs, hyps, bleu_scores = [], [], []

print(f"🔹 Evaluating quietly on {num_eval} random samples...")

for i in np.random.choice(len(enc_test), num_eval, replace=False):
    inp = ' '.join([reverse_word_index.get(idx, '') for idx in enc_test[i] if idx > 0])
    ref = ' '.join([reverse_word_index.get(idx, '') for idx in dec_test[i] if idx > 0])
    hyp = decode_sequence(inp)
    refs.append(ref.split())
    hyps.append(hyp.split())
    bleu = sentence_bleu([ref.split()], hyp.split(), smoothing_function=smoothie)
    bleu_scores.append(bleu)

# --- Results ---
avg_bleu = np.mean(bleu_scores)
print(f"\n✅ Average BLEU score over {num_eval} samples: {avg_bleu:.4f}")

print("\n🧩 Sample qualitative outputs:\n")
for j in range(5):
    print(f"Example {j+1}")
    print(f"Input:     {' '.join([reverse_word_index.get(idx, '') for idx in enc_test[j] if idx > 0])}")
    print(f"Reference: {' '.join(refs[j])}")
    print(f"Predicted: {' '.join(hyps[j])}")
    print(f"BLEU:      {bleu_scores[j]:.3f}\n")


🔹 Evaluating quietly on 250 random samples...

✅ Average BLEU score over 250 samples: 0.0129

🧩 Sample qualitative outputs:

Example 1
Input:     if unicorns were real the demand for would go down clearly
Reference: <start> that's really cool! i didn't know about that <end>
Predicted: i think i would be able to go to the advertising
BLEU:      0.022

Example 2
Input:     or thats how many his count went up in the last few minutes
Reference: <start> because i have been chatting for over five hours already <end>
Predicted: i think i would be a advertising
BLEU:      0.016

Example 3
Input:     not at people, though. don't get any kind of bad ideas about me with hand guns
Reference: <start> shes like and she is the first climber ever to climb a grade <end>
Predicted: i think i was like a advertising
BLEU:      0.012

Example 4
Input:     so i just use it for everything
Reference: <start> yep <end>
Predicted: i am doing well!
BLEU:      0.000

Example 5
Input:     like isn't that the oppos